In [1]:
%load_ext autoreload
%autoreload 2

In [14]:
import os 
import sys
project_dir_path = '/home/onoue/ws/lukasiewicz_1'
sys.path.append(project_dir_path)

import cvxpy as cp
import numpy as np 
import pandas as pd


from src.setup_problem_primal_modular import Setup
from src.objective_function import specimen_construct_objective_function, specimen_construct_objective_function_loss
from src.evaluation import EvaluateModel


In [10]:
import os
import sys
project_dir_path = '/home/onoue/ws/lukasiewicz_1'
sys.path.append(project_dir_path)
import shutil

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split


# 提案アルゴリズム用データフォーマット

- 実験設定（seed, その他）

In [11]:
data = pd.read_csv("./data/diabetes_discretized.csv", index_col=0)
X = data.drop(["Outcome"], axis=1)
y = data["Outcome"]

random_state = 42
test_size = 0.2

X_train, X_test, _, _ = train_test_split(X, y,
                                         test_size=test_size,
                                         random_state=random_state)

train_index = X_train.index
test_index = X_test.index


train_data = data.loc[train_index, :]
outcome = train_data['Outcome']
features = train_data.drop(['Outcome'], axis=1)
feature_names = list(features.columns)


train_data_dir_path = os.path.join(project_dir_path, 'inputs/pima_indian_diabetes/train')

if not os.path.exists(train_data_dir_path):
    os.mkdir(train_data_dir_path)


df = features.copy()
df['target'] = outcome.replace(0, -1)
display(df)

file_name = "L_" + "Outcome" + '.csv'
file_path = os.path.join(train_data_dir_path, file_name)
df.to_csv(file_path)


for feature_name in feature_names:
    df = features.copy()
    df['target'] = df[feature_name].replace(0, -1)
    # display(df)

    file_name = "L_" + feature_name + '.csv'
    file_path = os.path.join(train_data_dir_path, file_name)
    df.to_csv(file_path)


data_num = 15
data_dim = 21

arr_U = np.random.randint(2, size=(data_num, data_dim))
df_U = pd.DataFrame(arr_U)
df_U.to_csv(os.path.join(train_data_dir_path, 'U.csv'))

# rule.txt
import shutil
source_path = './data/rules.txt'
destination_path = os.path.join(train_data_dir_path, "rules.txt")
shutil.copy(source_path, destination_path)


test_data = data.loc[test_index, :]

outcome = test_data['Outcome']
features = test_data.drop(['Outcome'], axis=1)
feature_names = list(features.columns)


test_data_dir_path = os.path.join(project_dir_path, 'inputs/pima_indian_diabetes/test')

if not os.path.exists(test_data_dir_path):
    os.mkdir(test_data_dir_path)

 
df = features.copy()
df['target'] = outcome.replace(0, -1)
display(df)

file_name = "L_" + "Outcome" + '.csv'
file_path = os.path.join(test_data_dir_path, file_name)
df.to_csv(file_path)


,Pregnancies_Low,Pregnancies_Medium,Pregnancies_High,Glucose_Low,Glucose_Medium,Glucose_High,BloodPressure_Low,BloodPressure_Medium,BloodPressure_High,SkinThickness_Low,...,BMI_Low,BMI_Medium,BMI_High,DiabetesPedigreeFunction_Low,DiabetesPedigreeFunction_Medium,DiabetesPedigreeFunction_High,Age_Low,Age_Medium,Age_High,target
275,1,0,0,0,1,0,0,1,0,0,...,0,0,1,0,1,0,1,0,0,-1
555,0,1,0,0,1,0,0,1,0,0,...,1,0,0,1,0,0,0,1,0,-1
699,1,0,0,0,1,0,0,1,0,1,...,0,0,1,0,0,1,1,0,0,-1
73,1,0,0,0,1,0,0,0,1,1,...,0,1,0,1,0,0,1,0,0,-1
170,0,1,0,0,1,0,0,1,0,1,...,0,1,0,1,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,1,0,0,0,1,0,0,1,0,0,...,0,1,0,1,0,0,1,0,0,-1
128,1,0,0,0,1,0,0,0,1,0,...,0,1,0,1,0,0,0,1,0,1
315,1,0,0,0,1,0,0,1,0,0,...,0,1,0,1,0,0,1,0,0,-1
505,0,0,1,1,0,0,0,1,0,1,...,0,1,0,1,0,0,0,1,0,-1


'/home/onoue/ws/lukasiewicz_1/inputs/pima_indian_diabetes/train/rules.txt'

In [15]:
data_dir_path = os.path.join(project_dir_path, 'inputs/pima_indian_diabetes')
train_data_dir_path = os.path.join(data_dir_path, "train")
file_list = os.listdir(train_data_dir_path)

L_files = [filename for filename in file_list 
           if filename.startswith('L') and filename.endswith('.csv')]

U_files = [filename for filename in file_list 
           if filename.startswith('U') and filename.endswith('.csv')]

file_names_dict = {
    'supervised': L_files,
    'unsupervised': U_files,
    'rule': ['rules.txt']
}


obj_constructor = specimen_construct_objective_function
problem_instance = Setup(train_data_dir_path, file_names_dict, obj_constructor)
objective, constraints = problem_instance.main(c1=10, c2=10)

problem = cp.Problem(objective, constraints)
result = problem.solve(verbose=True)

load_data took 0.11553645133972168 seconds!
load_rules took 0.0005080699920654297 seconds!
specimen_construct_objective_function took 4.653881788253784 seconds!
pointwise constraints
logical constraints
consistency constraints
__call__ took 21.220083475112915 seconds!


/home/onoue/ws/lukasiewicz_1/myenv/lib/python3.10/site-packages/cvxpy/problems/problem.py:157: UserWarning: Objective contains too many subexpressions. Consider vectorizing your CVXPY code to speed up compilation.
  warnings.warn("Objective contains too many subexpressions. "


                                     CVXPY                                     
                                     v1.3.2                                    
(CVXPY) Dec 23 04:23:41 PM: Your problem has 12322 variables, 37416 constraints, and 0 parameters.
(CVXPY) Dec 23 04:23:45 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Dec 23 04:23:45 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Dec 23 04:23:45 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Dec 23 04:23:50 PM: Compiling problem (target solver=ECOS).
(CVXPY) Dec 23 04:23:50 PM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuf

# 表埋め 手順

- 元データファイル： diabetes_cleaned.csv, diabetes_discretized.csv

In [6]:
from src.evaluation import EvaluateModel

In [16]:
problem_instance.L

{'Age_Medium': array([[ 1,  0,  0, ...,  0,  0, -1],
        [ 0,  1,  0, ...,  1,  0,  1],
        [ 1,  0,  0, ...,  0,  0, -1],
        ...,
        [ 1,  0,  0, ...,  0,  0, -1],
        [ 0,  0,  1, ...,  1,  0,  1],
        [ 1,  0,  0, ...,  0,  0, -1]]),
 'DiabetesPedigreeFunction_Low': array([[ 1,  0,  0, ...,  0,  0, -1],
        [ 0,  1,  0, ...,  1,  0,  1],
        [ 1,  0,  0, ...,  0,  0, -1],
        ...,
        [ 1,  0,  0, ...,  0,  0,  1],
        [ 0,  0,  1, ...,  1,  0,  1],
        [ 1,  0,  0, ...,  0,  0,  1]]),
 'BMI_Low': array([[ 1,  0,  0, ...,  0,  0, -1],
        [ 0,  1,  0, ...,  1,  0,  1],
        [ 1,  0,  0, ...,  0,  0, -1],
        ...,
        [ 1,  0,  0, ...,  0,  0, -1],
        [ 0,  0,  1, ...,  1,  0, -1],
        [ 1,  0,  0, ...,  0,  0, -1]]),
 'BMI_Medium': array([[ 1,  0,  0, ...,  0,  0, -1],
        [ 0,  1,  0, ...,  1,  0, -1],
        [ 1,  0,  0, ...,  0,  0, -1],
        ...,
        [ 1,  0,  0, ...,  0,  0,  1],
        [ 0, 

In [17]:
import numpy as np

In [18]:
x1 = np.random.rand(1, 2)
x2 = np.random.rand(2, 10)

np.dot(x1, x2)

array([[0.59283515, 0.33510214, 0.63797869, 0.38252144, 0.58713692,
        0.79645477, 0.29739947, 0.86152218, 0.98238194, 0.53470259]])